In [20]:
#导入相关的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
#from pandas_profiling import ProfileReport
#import missingno as msno
from scipy.sparse import csr_matrix
from sklearn.model_selection import KFold, StratifiedKFold
%matplotlib inline

In [21]:
#导入训练集
train = pd.read_csv('train_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [22]:
#导入测试集
test = pd.read_csv('test_set.csv')
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [23]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse

test['y']=-1

cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

feature1 = cate_features + num_features
data = pd.concat([train,test])
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [24]:
# 特征工程1

#def feat_count(df, df_feature,feature,value,name=""):
    #df_count = pd.DataFrame(df_feature.groupby(feature)[value].count()).reset_index()
    #if not name:
        #df_count.columns = feature + [value+"_%s_count" % ("_".join(feature[0]))] 
    #else:
        #df_count.columns = feature + [name] # 注意这里表示df_count增加一列，且列名为[name]，上一行代码同理
    #df = df.merge(df_count, on=feature, how="left")#.fillna(0)
    #return df,df_count.columns[1]

#for fe in feature1:
    #data = feat_count(data,data,['ID'],fe)[0] # feat_count函数返回2个值，【0】表示第一个值

#data.head()

In [25]:
# 特征工程2

def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name


ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)

In [26]:
# 特征工程3

sparse_feature= ['contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']

def get_new_columns(name,aggs):
    l=[]
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg))=="<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    for den in dense_feature:
        aggs[den]=['mean','max','min','std']
    aggs.pop(d)
    temp=data.groupby(d).agg(aggs).reset_index()
    temp.columns=[d]+get_new_columns(d,aggs)
    data=pd.merge(data,temp,on=d,how='left')

100%|██████████| 9/9 [00:23<00:00,  4.12s/it]


In [27]:
#num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

In [28]:
# 获取数值特征

feature_all_ori = feature1 + ['ID','y']
num_features_add = []
for feature in data.columns.tolist():
    if feature not in feature_all_ori:
        num_features_add.append(feature)
        
num_features += num_features_add
num_features

['age',
 'balance',
 'day',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'contact_default_count',
 'contact_default_nunique',
 'contact_education_count',
 'contact_education_nunique',
 'contact_housing_count',
 'contact_housing_nunique',
 'contact_job_count',
 'contact_job_nunique',
 'contact_loan_count',
 'contact_loan_nunique',
 'contact_marital_count',
 'contact_marital_nunique',
 'contact_month_count',
 'contact_month_nunique',
 'contact_poutcome_count',
 'contact_poutcome_nunique',
 'contact_age_mean',
 'contact_age_max',
 'contact_age_min',
 'contact_age_std',
 'contact_balance_mean',
 'contact_balance_max',
 'contact_balance_min',
 'contact_balance_std',
 'contact_day_mean',


In [29]:
# 数据标准化

from sklearn.preprocessing import StandardScaler, MinMaxScaler
mms = StandardScaler()
data[num_features] = mms.fit_transform(data[num_features])

G:\AIAIAI\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
G:\AIAIAI\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [30]:
# 获取独热特征

one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']

In [31]:
# 获取向量化特征

data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))

vector_feature = ['new_con']

In [32]:
for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

In [33]:
train=data[data.y!=-1]
test=data[data.y==-1]
res=test[['ID']]
train_x=train[num_features]
test_x=test[num_features]

In [34]:
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a= enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

G:\AIAIAI\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
G:\AIAIAI\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this 

In [35]:
cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

In [36]:
import lightgbm as lgb
n_splits=5
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
    boosting_type="gbdt",
    num_leaves=30,
    reg_alpha=0,
    reg_lambda=0.,
    max_depth=8,
    n_estimators=1500,
    objective='binary',
    metric= 'auc',
    subsample=0.95,
    colsample_bytree=0.7,
    subsample_freq=1,
    learning_rate=0.008,
    random_state=2017
    )

train_y=train['y']
res['pred'] = 0

G:\AIAIAI\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [37]:
from sklearn.metrics import roc_auc_score
start = time()
pred_valid = pd.DataFrame()

for train_idx, val_idx in kfold.split(train_x, train_y):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    valid_x1 = train_x[val_idx]
    valid_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(valid_x1, valid_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]
    pred_valid1 = pd.DataFrame({'Id':valid_y1.index, 'pred':clf.predict_proba(valid_x1)[:,1], 'y':valid_y1.values})
    pred_valid = pred_valid.append(pred_valid1).reset_index(drop=True)
    
end = time()

auc = roc_auc_score(pred_valid['y'], pred_valid['pred'])
print('验证集得分为：{}'.format(auc))
print('训练时间为：{} min'.format((end-start)/60))

[1]	training's auc: 0.898342	valid_1's auc: 0.887259
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.926266	valid_1's auc: 0.912638
[3]	training's auc: 0.929057	valid_1's auc: 0.91429
[4]	training's auc: 0.932694	valid_1's auc: 0.921536
[5]	training's auc: 0.932021	valid_1's auc: 0.920485
[6]	training's auc: 0.934033	valid_1's auc: 0.922551
[7]	training's auc: 0.932681	valid_1's auc: 0.919496
[8]	training's auc: 0.932053	valid_1's auc: 0.917788
[9]	training's auc: 0.933986	valid_1's auc: 0.920329
[10]	training's auc: 0.933003	valid_1's auc: 0.919573
[11]	training's auc: 0.934167	valid_1's auc: 0.920888
[12]	training's auc: 0.936996	valid_1's auc: 0.924178
[13]	training's auc: 0.93629	valid_1's auc: 0.923425
[14]	training's auc: 0.935612	valid_1's auc: 0.922228
[15]	training's auc: 0.934446	valid_1's auc: 0.920852
[16]	training's auc: 0.936304	valid_1's auc: 0.92263
[17]	training's auc: 0.937572	valid_1's auc: 0.923795
[18]	training's auc: 0.93894	va

[151]	training's auc: 0.95028	valid_1's auc: 0.933021
[152]	training's auc: 0.95036	valid_1's auc: 0.933028
[153]	training's auc: 0.950387	valid_1's auc: 0.932959
[154]	training's auc: 0.950437	valid_1's auc: 0.932901
[155]	training's auc: 0.950512	valid_1's auc: 0.932949
[156]	training's auc: 0.950573	valid_1's auc: 0.932953
[157]	training's auc: 0.950646	valid_1's auc: 0.932988
[158]	training's auc: 0.950681	valid_1's auc: 0.932926
[159]	training's auc: 0.950761	valid_1's auc: 0.932921
[160]	training's auc: 0.950801	valid_1's auc: 0.932934
[161]	training's auc: 0.95086	valid_1's auc: 0.932951
[162]	training's auc: 0.950912	valid_1's auc: 0.933089
[163]	training's auc: 0.950973	valid_1's auc: 0.93307
[164]	training's auc: 0.951036	valid_1's auc: 0.93305
[165]	training's auc: 0.9511	valid_1's auc: 0.933107
[166]	training's auc: 0.95114	valid_1's auc: 0.933076
[167]	training's auc: 0.951187	valid_1's auc: 0.933122
[168]	training's auc: 0.95123	valid_1's auc: 0.933085
[169]	training's au

[300]	training's auc: 0.957414	valid_1's auc: 0.934465
[301]	training's auc: 0.957442	valid_1's auc: 0.93445
[302]	training's auc: 0.957479	valid_1's auc: 0.93446
[303]	training's auc: 0.957501	valid_1's auc: 0.93447
[304]	training's auc: 0.957535	valid_1's auc: 0.934525
[305]	training's auc: 0.957569	valid_1's auc: 0.934498
[306]	training's auc: 0.957608	valid_1's auc: 0.934503
[307]	training's auc: 0.957651	valid_1's auc: 0.934496
[308]	training's auc: 0.957694	valid_1's auc: 0.934502
[309]	training's auc: 0.957726	valid_1's auc: 0.934523
[310]	training's auc: 0.957768	valid_1's auc: 0.934528
[311]	training's auc: 0.957807	valid_1's auc: 0.934539
[312]	training's auc: 0.957844	valid_1's auc: 0.934531
[313]	training's auc: 0.957894	valid_1's auc: 0.934534
[314]	training's auc: 0.957937	valid_1's auc: 0.934559
[315]	training's auc: 0.957971	valid_1's auc: 0.934556
[316]	training's auc: 0.957997	valid_1's auc: 0.934564
[317]	training's auc: 0.958041	valid_1's auc: 0.93456
[318]	training

[449]	training's auc: 0.962776	valid_1's auc: 0.935614
[450]	training's auc: 0.962801	valid_1's auc: 0.935608
[451]	training's auc: 0.962822	valid_1's auc: 0.935647
[452]	training's auc: 0.962862	valid_1's auc: 0.935658
[453]	training's auc: 0.962906	valid_1's auc: 0.935682
[454]	training's auc: 0.962937	valid_1's auc: 0.935671
[455]	training's auc: 0.962964	valid_1's auc: 0.935672
[456]	training's auc: 0.963	valid_1's auc: 0.935672
[457]	training's auc: 0.963039	valid_1's auc: 0.935686
[458]	training's auc: 0.963079	valid_1's auc: 0.935683
[459]	training's auc: 0.96312	valid_1's auc: 0.935723
[460]	training's auc: 0.963164	valid_1's auc: 0.935746
[461]	training's auc: 0.963198	valid_1's auc: 0.935765
[462]	training's auc: 0.963245	valid_1's auc: 0.93578
[463]	training's auc: 0.963277	valid_1's auc: 0.935766
[464]	training's auc: 0.96331	valid_1's auc: 0.93577
[465]	training's auc: 0.96334	valid_1's auc: 0.935768
[466]	training's auc: 0.963376	valid_1's auc: 0.935794
[467]	training's a

[598]	training's auc: 0.967536	valid_1's auc: 0.936224
[599]	training's auc: 0.967573	valid_1's auc: 0.936247
[600]	training's auc: 0.967605	valid_1's auc: 0.936255
[601]	training's auc: 0.967641	valid_1's auc: 0.936251
[602]	training's auc: 0.967672	valid_1's auc: 0.936264
[603]	training's auc: 0.96771	valid_1's auc: 0.936268
[604]	training's auc: 0.967731	valid_1's auc: 0.936263
[605]	training's auc: 0.967757	valid_1's auc: 0.93627
[606]	training's auc: 0.967798	valid_1's auc: 0.936261
[607]	training's auc: 0.967827	valid_1's auc: 0.93626
[608]	training's auc: 0.967858	valid_1's auc: 0.936265
[609]	training's auc: 0.96788	valid_1's auc: 0.936252
[610]	training's auc: 0.967908	valid_1's auc: 0.936269
[611]	training's auc: 0.967936	valid_1's auc: 0.936269
[612]	training's auc: 0.967971	valid_1's auc: 0.936293
[613]	training's auc: 0.968004	valid_1's auc: 0.936284
[614]	training's auc: 0.968047	valid_1's auc: 0.936294
[615]	training's auc: 0.968076	valid_1's auc: 0.936296
[616]	training

...

[469]	training's auc: 0.962161	valid_1's auc: 0.942735
[470]	training's auc: 0.962192	valid_1's auc: 0.942733
[471]	training's auc: 0.962226	valid_1's auc: 0.942734
[472]	training's auc: 0.962253	valid_1's auc: 0.942738
[473]	training's auc: 0.962286	valid_1's auc: 0.942738
[474]	training's auc: 0.962324	valid_1's auc: 0.942733
[475]	training's auc: 0.962351	valid_1's auc: 0.942747
[476]	training's auc: 0.962381	valid_1's auc: 0.942762
[477]	training's auc: 0.962409	valid_1's auc: 0.942746
[478]	training's auc: 0.962439	valid_1's auc: 0.942742
[479]	training's auc: 0.962482	valid_1's auc: 0.942745
[480]	training's auc: 0.962511	valid_1's auc: 0.942756
[481]	training's auc: 0.96254	valid_1's auc: 0.942759
[482]	training's auc: 0.962571	valid_1's auc: 0.942778
[483]	training's auc: 0.962623	valid_1's auc: 0.942775
[484]	training's auc: 0.96265	valid_1's auc: 0.942771
[485]	training's auc: 0.962693	valid_1's auc: 0.942786
[486]	training's auc: 0.962731	valid_1's auc: 0.942803
[487]	traini

[618]	training's auc: 0.967012	valid_1's auc: 0.943438
[619]	training's auc: 0.967034	valid_1's auc: 0.943442
[620]	training's auc: 0.967065	valid_1's auc: 0.943435
[621]	training's auc: 0.967106	valid_1's auc: 0.943438
[622]	training's auc: 0.967133	valid_1's auc: 0.943434
[623]	training's auc: 0.967169	valid_1's auc: 0.94345
[624]	training's auc: 0.967198	valid_1's auc: 0.943447
[625]	training's auc: 0.967236	valid_1's auc: 0.943469
[626]	training's auc: 0.967268	valid_1's auc: 0.943464
[627]	training's auc: 0.967314	valid_1's auc: 0.943474
[628]	training's auc: 0.967351	valid_1's auc: 0.943476
[629]	training's auc: 0.967386	valid_1's auc: 0.943474
[630]	training's auc: 0.967416	valid_1's auc: 0.943476
[631]	training's auc: 0.967438	valid_1's auc: 0.94349
[632]	training's auc: 0.967476	valid_1's auc: 0.943483
[633]	training's auc: 0.967512	valid_1's auc: 0.943488
[634]	training's auc: 0.967553	valid_1's auc: 0.943499
[635]	training's auc: 0.967585	valid_1's auc: 0.943474
[636]	traini

[767]	training's auc: 0.971422	valid_1's auc: 0.943788
[768]	training's auc: 0.971443	valid_1's auc: 0.943774
[769]	training's auc: 0.971468	valid_1's auc: 0.943775
[770]	training's auc: 0.971497	valid_1's auc: 0.943766
[771]	training's auc: 0.971527	valid_1's auc: 0.943784
[772]	training's auc: 0.971556	valid_1's auc: 0.94378
[773]	training's auc: 0.971585	valid_1's auc: 0.943756
[774]	training's auc: 0.971612	valid_1's auc: 0.943757
[775]	training's auc: 0.971648	valid_1's auc: 0.94376
[776]	training's auc: 0.971677	valid_1's auc: 0.943762
[777]	training's auc: 0.971703	valid_1's auc: 0.943752
[778]	training's auc: 0.971728	valid_1's auc: 0.943731
[779]	training's auc: 0.971755	valid_1's auc: 0.943744
[780]	training's auc: 0.97179	valid_1's auc: 0.943757
[781]	training's auc: 0.971819	valid_1's auc: 0.943757
[782]	training's auc: 0.971845	valid_1's auc: 0.943774
[783]	training's auc: 0.971873	valid_1's auc: 0.943767
[784]	training's auc: 0.971902	valid_1's auc: 0.943777
[785]	trainin

[916]	training's auc: 0.975195	valid_1's auc: 0.943862
[917]	training's auc: 0.975218	valid_1's auc: 0.943882
[918]	training's auc: 0.975244	valid_1's auc: 0.94388
[919]	training's auc: 0.975269	valid_1's auc: 0.943891
[920]	training's auc: 0.97529	valid_1's auc: 0.943893
[921]	training's auc: 0.975309	valid_1's auc: 0.943902
[922]	training's auc: 0.975333	valid_1's auc: 0.943921
[923]	training's auc: 0.975358	valid_1's auc: 0.943929
[924]	training's auc: 0.975379	valid_1's auc: 0.943921
[925]	training's auc: 0.975406	valid_1's auc: 0.94392
[926]	training's auc: 0.975437	valid_1's auc: 0.943911
[927]	training's auc: 0.975459	valid_1's auc: 0.943911
[928]	training's auc: 0.97549	valid_1's auc: 0.943912
[929]	training's auc: 0.975506	valid_1's auc: 0.943907
[930]	training's auc: 0.975527	valid_1's auc: 0.943917
[931]	training's auc: 0.975548	valid_1's auc: 0.943907
[932]	training's auc: 0.975566	valid_1's auc: 0.943902
[933]	training's auc: 0.975593	valid_1's auc: 0.943901
[934]	training

[1064]	training's auc: 0.97833	valid_1's auc: 0.943775
[1065]	training's auc: 0.978345	valid_1's auc: 0.943776
[1066]	training's auc: 0.978368	valid_1's auc: 0.943774
[1067]	training's auc: 0.97839	valid_1's auc: 0.943797
[1068]	training's auc: 0.978413	valid_1's auc: 0.943774
[1069]	training's auc: 0.978431	valid_1's auc: 0.943781
[1070]	training's auc: 0.978444	valid_1's auc: 0.943793
[1071]	training's auc: 0.978461	valid_1's auc: 0.943797
[1072]	training's auc: 0.978477	valid_1's auc: 0.943797
[1073]	training's auc: 0.978499	valid_1's auc: 0.943795
[1074]	training's auc: 0.978509	valid_1's auc: 0.94381
[1075]	training's auc: 0.978529	valid_1's auc: 0.943805
[1076]	training's auc: 0.978551	valid_1's auc: 0.943786
[1077]	training's auc: 0.978577	valid_1's auc: 0.943786
[1078]	training's auc: 0.978595	valid_1's auc: 0.943774
Early stopping, best iteration is:
[978]	training's auc: 0.976605	valid_1's auc: 0.943954
验证集得分为：0.9416232064128959
训练时间为：2.9322128931681317 min


In [38]:
res['pred'] = res['pred']/5
res.to_csv('bank.csv', index=False)

G:\AIAIAI\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
